In [1]:
import numpy as np

# Number of samples
N = 10**6

# Simulate data for H0
x0 = np.random.normal(0, 1, size=(N, 2))

# Simulate data for H1
x1_part1 = np.random.normal(1, 1, size=(N//2, 1))
x1_part2 = np.random.normal(-1, 1, size=(N//2, 1))
x1 = np.vstack([x1_part1, x1_part2])
x1 = np.hstack([x1, np.random.normal(0, 1, size=(N, 1))])

# Combine the data and create labels
X = np.vstack([x0, x1])
y = np.array([0]*N + [1]*N)


In [2]:
def bayes_decision_rule(x):
    f0 = np.exp(-0.5 * np.sum(x**2, axis=1))  # Likelihood under H0
    f1 = 0.5 * (np.exp(-0.5 * (x[:, 0] - 1)**2) + np.exp(-0.5 * (x[:, 0] + 1)**2)) * np.exp(-0.5 * x[:, 1]**2)  # Likelihood under H1
    return f1 > f0

# Apply the Bayes decision rule
y_bayes = bayes_decision_rule(X)


In [3]:
# Estimate error
error_rate_bayes = np.mean(y_bayes != y)
print("Bayes Decision Rule Error Rate:", error_rate_bayes)


Bayes Decision Rule Error Rate: 0.396392


In [5]:
# Function to generate data
def generate_data(N):
    # H0 data
    x0 = np.random.normal(0, 1, size=(N, 2))
    
    # H1 data
    x1_part1 = np.random.normal(1, 1, size=(N//2, 1))
    x1_part2 = np.random.normal(-1, 1, size=(N//2, 1))
    x1 = np.vstack([x1_part1, x1_part2])
    x1 = np.hstack([x1, np.random.normal(0, 1, size=(N, 1))])
    
    X = np.vstack([x0, x1])
    y = np.array([0]*N + [1]*N)
    
    return X, y

In [7]:
conda install pytorch torchvision torchaudio cpuonly -c pytorch


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\vanes\anaconda3

  added / updated specs:
    - cpuonly
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cpuonly-2.0                |                0           2 KB  pytorch
    libjpeg-turbo-2.0.0        |       h196d8e1_0         618 KB
    libuv-1.44.2               |       h2bbff1b_0         288 KB
    pytorch-2.3.0              |      py3.9_cpu_0       138.3 MB  pytorch
    pytorch-mutex-1.0          |              cpu           3 KB  pytorch
    torchaudio-2.3.0           |         py39_cpu         5.7 MB  pytorch
    torc



==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Generate the additional 200 samples for each hypothesis
X_train, y_train = generate_data(200)

# Convert to torch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1)

# Create a DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [9]:
# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 1)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


In [10]:
# Instantiate two networks, one for cross-entropy and one for exponential loss
net_ce = Net()
net_exp = Net()

# Define the loss functions
criterion_ce = nn.BCELoss()  # Binary cross-entropy
def exponential_loss(output, target):
    return torch.mean(0.5 * torch.exp(0.5 * (output - target)**2))

# Define the optimizers
optimizer_ce = optim.SGD(net_ce.parameters(), lr=0.01)
optimizer_exp = optim.SGD(net_exp.parameters(), lr=0.01)

In [17]:
# Training function
def train_network(net, criterion, optimizer, epochs=1000):
    for epoch in range(epochs):
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

In [33]:
# Train both networks
train_network(net_ce, criterion_ce, optimizer_ce)
train_network(net_exp, exponential_loss, optimizer_exp)

In [34]:
#Generate a large test dataset to evaluate performance
X_test, y_test = generate_data(10**6)
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float().view(-1, 1)

In [35]:
# Evaluate the networks on the test data
with torch.no_grad():
    outputs_ce = net_ce(X_test_tensor)
    outputs_exp = net_exp(X_test_tensor)

# Apply the decision rule: classify as H1 if output > 0.5
y_pred_ce = (outputs_ce > 0.5).float()
y_pred_exp = (outputs_exp > 0.5).float()

In [36]:
# Compute error rates
error_rate_ce = torch.mean((y_pred_ce != y_test_tensor).float())
error_rate_exp = torch.mean((y_pred_exp != y_test_tensor).float())

print(f'Cross-Entropy NN Error Rate: {error_rate_ce.item()}')
print(f'Exponential Loss NN Error Rate: {error_rate_exp.item()}')

Cross-Entropy NN Error Rate: 0.39997598528862
Exponential Loss NN Error Rate: 0.5012425184249878


In [37]:
# Compare with Bayes Decision Rule
def bayes_decision_rule(x):
    f0 = np.exp(-0.5 * np.sum(x**2, axis=1))
    f1 = 0.5 * (np.exp(-0.5 * (x[:, 0] - 1)**2) + np.exp(-0.5 * (x[:, 0] + 1)**2)) * np.exp(-0.5 * x[:, 1]**2)
    return f1 > f0

y_bayes = bayes_decision_rule(X_test)
error_rate_bayes = np.mean(y_bayes != y_test)
print(f'Bayes Decision Rule Error Rate: {error_rate_bayes}')

Bayes Decision Rule Error Rate: 0.396736
